In [ ]:
import base64
import requests
import time

def transcribe(audioStr):
    try:
        messages = [
            {
                "role": "system",
                "content": "You are a professional AI assistant specializing in ASR"
            },
            {
                "role": "user",
                "content": [
                    { "type": "audio_url", "audio_url": { "url": audioStr } },
                    { "type": "text", "text": "轉寫中文文本" }
                ]
            }
        ]

        # 请将以下 URL 替换为实际的 API 端点
        url = 'http://localhost:8000/speech2text'

        headers = {
            'Content-Type': 'application/json'
            # 如需身份验证，请在此处添加 'Authorization' 字段
        }

        payload = {
            'model': 'qwen2-audio-7b-instruct',
            'max_tokens': 256,
            'temperature': 0.001,
            'messages': messages
        }

        response = requests.post(url, headers=headers, json=payload)

        if not response.ok:
            raise Exception(f"HTTP error! status: {response.status_code}")

        data = response.json()

        return data

    except Exception as error:
        print('Error:', error)
        raise error

audio_path = '/Users/jon/Desktop/sample.mp3'

# 读取 MP3 文件的二进制数据
with open(audio_path, 'rb') as file:
        binary_data = file.read()
# 将二进制数据转换为 base64 编码的字符串
base64_encoded_data = base64.b64encode(binary_data).decode('utf-8')
data_uri = f'data:audio/mp3;base64,{base64_encoded_data}'
start_time = time.time()
segments = transcribe(data_uri)
print('Time:', time.time() - start_time)
print(segments)

{'segments': [{'start': 0.1, 'end': 9.38, 'words': [{'start': 0.1, 'end': 0.12, 'word': '好', 'confidence': 0.7518019676208496}, {'start': 0.22, 'end': 0.24, 'word': '被', 'confidence': 0.24829372763633728}, {'start': 0.38, 'end': 0.4, 'word': '撞', 'confidence': 0.8927834033966064}, {'start': 0.58, 'end': 0.6, 'word': '擊', 'confidence': 0.012735895812511444}, {'start': 0.7000000000000001, 'end': 0.72, 'word': '而', 'confidence': 0.8934530019760132}, {'start': 0.8, 'end': 0.8200000000000001, 'word': '不', 'confidence': 0.9935423135757446}, {'start': 0.9, 'end': 0.92, 'word': '幸', 'confidence': 0.07984600961208344}, {'start': 1.1, 'end': 1.12, 'word': '身', 'confidence': 0.8610385060310364}, {'start': 1.26, 'end': 1.28, 'word': '亡', 'confidence': 0.01957605592906475}, {'start': 1.3800000000000001, 'end': 1.42, 'word': '的', 'confidence': 0.8767673969268799}, {'start': 1.52, 'end': 1.54, 'word': '蘇', 'confidence': 0.8805347681045532}, {'start': 1.7, 'end': 1.72, 'word': '姓', 'confidence': 0.007